In [1]:
# sc.list_packages()
sc.install_pypi_package("pandas==0.25.1")
sc.install_pypi_package("boto3==1.26.7")
sc.install_pypi_package("rapidfuzz")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1679152595079_0002,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/9d/17/bc2db32cb4cd0649586e01f1958f5c21f85d3c28c6703e2dd8174b3d44a1/boto3-1.26.7-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/c7/fc/a3233f966da8df1a580a8e0e78e0be2adc57ad97c10aac303b15b7dbd0ac/botocore-1.29.94-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/5e/c6/af903b5fab3f9b5b1e883f49a770066314c6dcceb589cf938d48c89556c1/s3transfer-0.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7b/f5/890a0baca17a61c1f92f72b81d3c31523c99bec609e60c292ea55b387ae8/urllib3-1.26.15-py2.py3-none-any.whl


You are using pip version 9.0.1, however

In [2]:
confs = ["spark.yarn.executor.memoryOverhead","spark.sql.shuffle.partitions","spark.network.timeout","spark.driver.memory"]
print(sc._conf.get('spark.app.name'))
print(sc._conf.get('spark.submit.deployMode'))
print(sc._conf.get("spark.yarn.executor.memoryOverhead"))
print(sc._conf.get("spark.sql.files.maxPartitionBytes"))
for conf in confs:
    print(conf,":",sc._conf.get(conf))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

livy-session-1
client
None
None
spark.yarn.executor.memoryOverhead : None
spark.sql.shuffle.partitions : None
spark.network.timeout : None
spark.driver.memory : 2048M

# Imports

In [3]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import year, month, dayofmonth
import boto3
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType, MapType, IntegerType, DoubleType
from pyspark.sql.types import *
from pyspark.sql import functions as F
from collections import Counter
import pyspark
import rapidfuzz
from rapidfuzz import fuzz
import time
import random

# import s3fs
# import boto3
#How to pip install while creation of cluster

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:

spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Task #43

1. Find out the files which contain mention of BOTH US and China state, province or country.
2. Check for the presence of broad themes and also check for the presence of sub-theme under each umbrella theme.
3. Report the article count, average tone for each parent as well as sub theme.



## 1.  Reading Files

In [5]:
from boto3 import client

def return_yearly_parquet_files(year):
    print(f"Return file list for the year: {year}")
    file_list = []
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='kcsra', Prefix = f"{str(year)} GDELT GKG PG")['Contents']:
        file = "s3://kcsra/"+ key['Key']
        if "parquet" in file:
            file_list.append(file)
    return file_list

def folder_parquet_files(year):
    print(f"Return file list for the year: {year}")
    file_list = []
    conn = client('s3')  # again assumes boto.cfg setup, assume AWS S3
    for key in conn.list_objects(Bucket='kcsra', Prefix = f"{str(year)} GDELT GKG PG")['Contents']:
        file = "s3://kcsra/"+ key['Key']
        if "parquet" in file:
            file_list.append(file)
    return file_list


def return_correct_file_paths(year_file_list):
    print("Returning the working parquet files ...")
    correct_file_paths = []
    for i,file in enumerate(year_file_list):

        try:
            df = spark.read.parquet(file)
            correct_file_paths.append(file)
        except Exception as e:
            print(f"[{str(i+1)}/{len(year_file_list)}]")
            print("->ERROR while reading:")
            print(file)
            print()
    return correct_file_paths

def return_df(correct_file_paths, req_cols, grid_id = None, prio_grid_list = None):
    print("Constructing a dataframe from required file paths")

    df = spark.read.parquet(*correct_file_paths).select(*req_cols).dropDuplicates(['gkgrecordid'])
    
    if grid_id:
        print("Filter Grid ID:", grid_id)
        df = df.filter(df.GRID_IDs == grid_id)
    
    if prio_grid_list:
        print("Filter Prio Grid ID List:", prio_grid_list)
#         print(type(prio_grid_list))
        df = df.filter(df.GRID_IDs.isin(prio_grid_list))

    df = df.withColumn('day',dayofmonth(df.date)).withColumn('month',month(df.date))
    df = df.dropna(subset = req_cols)
    print("Total rows:",df.count())
    # df.printSchema()
    return df

def extract_tone(tone_string):
    try:
        tone_float = re.findall("tone=(.*?),",str(tone_string))[0].strip("'")
        tone_float = float(tone_float)
        return tone_float
    except Exception as e:
        print("Error extracting tone:" + str(e))
        print(tone_string)
        print("="*20)
        return None
    
def save_tone_in_new_col(df):
    print("Saving the tone in a new column")
    extract_tone_udf = udf(extract_tone, DoubleType())
    df = df.withColumn('Extracted_tone', extract_tone_udf(df['tone'])).drop("tone").withColumnRenamed("Extracted_tone", "tone")
    return df

@udf(returnType=ArrayType(StringType()))
def retrieve_theme_list(themes):
    if not themes:
        return {}
    themes = str(themes)
    return list(Counter(re.findall(f"theme=(.*?),",themes)).keys())

@udf(returnType=ArrayType(StringType())) 
def retrieve_organization_list(organizations):
    if not organizations:
        return {}
    organizations = str(organizations)
    return list(Counter(re.findall(f"organization=(.*?),",organizations)).keys())

@udf(returnType=StringType())
def aggregate_category(category):
    super_counter = Counter({})
    for row in category:
        super_counter += Counter(row)
    return ", ".join([f"{categ[0]}:{categ[1]}" for categ in super_counter.most_common(20)])


def are_strings_similar(target_str, data_str, match_threshold):
    target_str, data_str = str(target_str).lower(), str(data_str).lower()
    partial_match, sorted_match = fuzz.partial_ratio(target_str, data_str), fuzz.token_sort_ratio(target_str, data_str)

    if max(partial_match, sorted_match) >= match_threshold:
        # print(max(partial_match, sorted_match))
        return True
    else:
        return False


def compare_string_matches_lists(target_list, data_list, match_threshold = 75):
    for target_str in target_list:
        for data_str in data_list:
            if are_strings_similar(target_str, data_str, match_threshold):
                # print("Match Found", f"| {target_str} : {data_str}")
                return 1
    return 0


def udf_compare_strings_lists(target_list, match_threshold):
    return udf(lambda data_list: compare_string_matches_lists(target_list, data_list, match_threshold))

@udf(returnType=IntegerType()) 
def check_presence_of_main_theme(*sub_theme_cols):
    presence = 0
    for sub_theme_col_count in sub_theme_cols:
        print("Sub theme col count")
        presence += sub_theme_col_count
    
    if presence > 0:
        return 1
    else:
        return 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
themes_dict = {'Political': ['interstate conflict', 'nationalism'],
 'Economic': ['economic integration',
  'governance uncertainty',
  'climate response',
  'economic stagnation',
  'pandemic response'],
 'Tech': ['innovation', 'green technology', 'digitalization']}

all_themes = list(themes_dict.keys())

for v in list(themes_dict.values()):
    all_themes.extend(v)
all_themes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Political', 'Economic', 'Tech', 'interstate conflict', 'nationalism', 'economic integration', 'governance uncertainty', 'climate response', 'economic stagnation', 'pandemic response', 'innovation', 'green technology', 'digitalization']

# MAIN

In [7]:
# match_str = """[Google, Ballistic Research Centre, Institute Of Forensic Sciences, Torrent Pharmaceuticals, Bajaj Alliance, Directorate Of Forensic Sciences, Institute Of Behavioral Sciences, Gujarat Forensic Sciences University, Formula Group, Institute Of Research Development, Congress Mp Shashi Tharoor, Telecom Day, Us Central Intelligence Agency, Cadila Pharmaceuticals, Directorate Of Forensic Science Laboratory]"""

# data_list = [x.strip() for x in match_str.strip().strip("[").strip("]").split(",")]
# compare_string_matches_lists(prim_firm_list, data_list, match_threshold = 75)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:

match_threshold = 80
req_cols = ['gkgrecordid','date','tone', 'year', 'themes']
t0 = time.time()


cols_list_ordering = ['year','month',"TotalArticles","AverageSentiment"]
for theme in all_themes:
    theme_total_articles = theme.replace(" ","_") + "_TotalArticles"
    average_sentiment_theme = "AverageSentiment_" + theme.replace(" ","_")
    cols_list_ordering.append(theme_total_articles)
    cols_list_ordering.append(average_sentiment_theme)
    
    
for year in range(2017, 2024):
    t1 = time.time()
    print(f"Year: {year}")
#     file_list = return_yearly_parquet_files(year)
#     file_list2 = [file for file in file_list if country in file]
    
    correct_file_paths = [f"s3://shrivats-dev/Data/#43_US_CH/{year}/"]
#     correct_file_paths = return_correct_file_paths(file_list2)
    df = return_df(correct_file_paths, req_cols)
    df = save_tone_in_new_col(df)
#     break
    df = df.dropna(subset = ["tone"])
    t2 = time.time()

    print("Time to read files:", t2-t1)
    df2 = df.withColumn("Theme_list", retrieve_theme_list(F.col("themes")))
    # df.unpersist()
    for k,v in themes_dict.items():
        for sub_theme in v:
            sub_theme_col_count = sub_theme.replace(" ","_")+"_count"
            sub_theme_col_tone = sub_theme.replace(" ","_")+"_tone"
            df2 = df2.withColumn(sub_theme_col_count, udf_compare_strings_lists([sub_theme], match_threshold)(F.col("Theme_list")).cast(IntegerType()))
            df2 = df2.withColumn(sub_theme_col_tone, F.col(sub_theme_col_count) * F.col("tone"))

        main_theme_col_count = k.replace(" ","_") + "_count"
        main_theme_col_tone = k.replace(" ","_") + "_tone"

        sub_theme_cols = [F.col(sub_theme.replace(" ","_")+"_count") for sub_theme in v] 
        df2 = df2.withColumn(main_theme_col_count,check_presence_of_main_theme(*sub_theme_cols))
        df2 = df2.withColumn(main_theme_col_tone, F.col(main_theme_col_count) * F.col("tone"))
    
    t3 = time.time()
    print("T3 time:", t3-t2)
   
    
    # df3.unpersist()
    aggregations = []
    for k,v in themes_dict.items():
        main_theme_col_count = k.replace(" ","_") + "_count"
        main_theme_col_tone = k.replace(" ","_") + "_tone"
        aggregations.append(F.sum(main_theme_col_count).alias(k.replace(" ","_") + "_TotalArticles"))
        aggregations.append(F.sum(main_theme_col_tone).alias(k.replace(" ","_") + "_ToneSum"))
        for sub_theme in v:
            sub_theme_col_count = sub_theme.replace(" ","_")+"_count"
            sub_theme_col_tone = sub_theme.replace(" ","_")+"_tone"
            aggregations.append(F.sum(sub_theme_col_count).alias(sub_theme.replace(" ","_") + "_TotalArticles"))
            aggregations.append(F.sum(sub_theme_col_tone).alias(sub_theme.replace(" ","_") + "_ToneSum"))
    
    print(aggregations)
    aggregation_cols = ['month','year']
    df3 = df2.groupby(*aggregation_cols).agg(F.count(F.col("tone")).alias('TotalArticles'),\
                              F.avg('tone').alias('AverageSentiment'),\
                              *aggregations)
    
    for theme in all_themes:
        theme_tone_sum = theme.replace(" ","_") + "_ToneSum"
        theme_total_articles = theme.replace(" ","_") + "_TotalArticles"
        average_sentiment_theme = "AverageSentiment_" + theme.replace(" ","_")
        df3 = df3.withColumn(average_sentiment_theme,F.col(theme_tone_sum)/F.col(theme_total_articles))\
                    .drop(theme_tone_sum)
    
    df3 = df3.fillna(value=0).select(*cols_list_ordering)
    t4 = time.time()
    print("T4 time:", t4-t3)

    df3.repartition(1).write.option("header",True) \
       .csv(f"s3://shrivats-dev/Tasks/#43_US_CH/Monthly/{year}")
    t5 = time.time()
    print("T5 time:", t5-t4)
    print("="*50)

#     break
print("Total time", time.time()-t0)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cols_list_ordering

## Joining Files

In [134]:
folder = "#43_US_CH"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### All

In [56]:
df_combined_all = spark.read.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/*/2*/*.csv")
df_combined_all = df_combined_all.withColumn("month", F.col("month").cast(IntegerType()))
df_combined_all_sorted = df_combined_all.sort("year","month")
df_combined_all_sorted.repartition(1).write.option("header",True) \
               .csv(f"s3://shrivats-dev/Tasks/{folder}/All")  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…